In [5]:
%load_ext autoreload
%autoreload 2
from tqdm import tqdm
import sys, os
sys.path.append("./ner")
from ner.train import *
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
from torch import FloatTensor, LongTensor
ids = [
    [1, 2, 3],
    [1, 2],
    [3, 4, 5, 5],
    [1, 1, 1],
    [10, 2, 3, 19, 3, 3, 33, 2, 19, 38, 3]
]

padded_tensor = torch.nn.utils.rnn.pad_sequence([FloatTensor(x) for x in ids], batch_first=True)
lengths = LongTensor([len(x) for x in ids])
sorted_lengths, parm_index = lengths.sort(descending=True)
pack_padded_sequence = torch.nn.utils.rnn.pack_padded_sequence(padded_tensor[parm_index], sorted_lengths, batch_first=True)

In [53]:
pack_padded_sequence

PackedSequence(data=tensor([10.,  3.,  1.,  1.,  1.,  2.,  4.,  2.,  1.,  2.,  3.,  5.,  3.,  1.,
        19.,  5.,  3.,  3., 33.,  2., 19., 38.,  3.]), batch_sizes=tensor([5, 5, 4, 2, 1, 1, 1, 1, 1, 1, 1]))

In [54]:
parm_index

tensor([4, 2, 0, 3, 1])

PackedSequence(data=tensor([10.,  3.,  1.,  1.,  1.,  2.,  4.,  2.,  1.,  2.,  3.,  5.,  3.,  1.,
        19.,  5.,  3.,  3., 33.,  2., 19., 38.,  3.]), batch_sizes=tensor([5, 5, 4, 2, 1, 1, 1, 1, 1, 1, 1]))

In [4]:
#Dataset読み込み
dataset = Dataset()
dataset.gpu = torch.cuda.is_available()
dataset.read_config("config/csw.4k_16k.201904121018.config")
if dataset.pretrain_model_dir:
    # pretrainを行ったdsetを読み込み、AlphabetをPretrainにする。
    lm_dset = Dataset()
    lm_dset.load(f"{dataset.pretrain_model_dir}.dset")
    dataset.load_lm_dset_alphabets(lm_dset)
    print("===============================")
    dataset.describe()

============= load config file ==================
train_dir=/home/sekine/SubwordSequenceLabeling/Repository/Chemdner/train.bioes: <class 'str'>
dev_dir=/home/sekine/SubwordSequenceLabeling/Repository/Chemdner/valid.bioes: <class 'str'>
test_dir=/home/sekine/SubwordSequenceLabeling/Repository/Chemdner/test.bioes: <class 'str'>
model_dir=/home/sekine/SubwordSequenceLabeling/Repository/NERModel/csw4k_16k.lm200k: <class 'str'>
lm_dir=/home/sekine/SubwordSequenceLabeling/Repository/LargeCorpus/large_corpus_200k.txt: <class 'str'>
pretrain_model_dir=/home/sekine/SubwordSequenceLabeling/Repository/LanguageModel/csw4k_16k.lm200k: <class 'str'>
word_emb_dir=/home/sekine/SubwordSequenceLabeling/Repository/GloVe/gv.d50: <class 'str'>
sw_num=2: <class 'int'>
sentence_piece_dirs=[/home/sekine/SubwordSequenceLabeling/Repository/SentencePiece/sp4000.model,/home/sekine/SubwordSequenceLabeling/Repository/SentencePiece/sp16000.model]: <class 'list'>
word_emb_dim=50: <class 'int'>
char_emb_dim=30: <class

903737it [00:00, 947738.81it/s]

=========== describe Dataset ===========
word_alphabet: 59281
subword_alphabet: [4155, 14599]
char_alphabet: 424
label_alphabet: 6


In [5]:
dataset.generate_instance('train')
dataset.generate_instance('dev')
dataset.generate_instance('test')

0it [00:00, ?it/s]

============== read train instance ====================


903737it [00:37, 23796.37it/s]
0it [00:00, ?it/s]

============== read dev instance ====================


898420it [00:40, 21995.21it/s]
3752it [00:00, 18756.28it/s]

============== read test instance ====================


776036it [00:36, 20975.67it/s]


In [52]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# 従来のNER ModelのTrain
model = SeqLabel(dataset)
optimizer = optim.SGD(model.parameters(), lr=dataset.lr, momentum=dataset.momentum,weight_decay=dataset.l2)
random.shuffle(dataset.train_Ids)
instance_count = 0
model.train()
model.zero_grad()
batch_size = dataset.batch_size
batch_id = 0
train_num = len(dataset.train_Ids)
total_batch = train_num//batch_size+1
for batch_id in tqdm(range(total_batch)):
    start = batch_id*batch_size
    end = (batch_id+1)*batch_size
    if end >train_num:
        end = train_num
    instance = dataset.train_Ids[start:end]
    if not instance:
        continue
    batch_word, batch_wordlen, batch_wordrecover, batch_char, batch_charlen, batch_charrecover, batch_sws, batch_swlens, batch_swrecovers, batch_swfmask, batch_swbmask, batch_label, mask = batchify_with_label(instance, dataset.gpu, True)
    instance_count += 1
    loss, tag_seq = model.neg_log_likelihood_loss(batch_word, batch_wordlen, batch_char, batch_charlen, batch_charrecover, batch_sws, batch_swlens, batch_swrecovers, batch_swfmask, batch_swbmask, batch_label, mask)
    print(loss)
    print(tag_seq)
    loss.backward()
    optimizer.step()
    model.zero_grad()

[SubwordRep(
  (drop): Dropout(p=0.5)
  (embeddings): Embedding(4155, 50)
  (f_lstm): LSTM(50, 25, batch_first=True)
  (b_lstm): LSTM(50, 25, batch_first=True)
), SubwordRep(
  (drop): Dropout(p=0.5)
  (embeddings): Embedding(14599, 50)
  (f_lstm): LSTM(50, 25, batch_first=True)
  (b_lstm): LSTM(50, 25, batch_first=True)
)]




  0%|          | 0/3081 [00:00<?, ?it/s]

build CRF...
tensor([[[ 0.1444, -0.0064, -0.1166,  ..., -0.0501, -0.1351, -0.1037],
         [ 0.1804, -0.0077, -0.1012,  ..., -0.0494, -0.2090, -0.0672],
         [ 0.1180, -0.0234, -0.0756,  ..., -0.0923, -0.1161,  0.0151],
         ...,
         [ 0.1707, -0.0026, -0.0967,  ..., -0.0322, -0.0780, -0.0518],
         [ 0.0565, -0.0431, -0.0539,  ..., -0.0020, -0.0393,  0.0001],
         [ 0.0484, -0.0103, -0.0331,  ..., -0.0568, -0.0859, -0.0176]],

        [[ 0.0436, -0.0132, -0.1208,  ..., -0.0001, -0.1154, -0.0458],
         [ 0.1067, -0.0469, -0.0756,  ..., -0.0731, -0.0886, -0.0045],
         [ 0.1457, -0.0316, -0.0768,  ..., -0.0831, -0.1742,  0.0015],
         ...,
         [ 0.0637,  0.0123, -0.0541,  ..., -0.0421, -0.0619, -0.0485],
         [ 0.0637,  0.0123, -0.0541,  ..., -0.0421, -0.0619, -0.0485],
         [ 0.0637,  0.0123, -0.0541,  ..., -0.0421, -0.0619, -0.0485]],

        [[ 0.0494,  0.0119,  0.0096,  ..., -0.0662, -0.0805, -0.0158],
         [ 0.0936,  0.0535, -0.0



  0%|          | 1/3081 [00:00<13:22,  3.84it/s]

tensor([[[ 0.0863,  0.0094, -0.0569,  ...,  0.0226, -0.1092, -0.0130],
         [ 0.1476, -0.0616, -0.0224,  ...,  0.0232, -0.1022, -0.0258],
         [ 0.0816,  0.0119, -0.0608,  ..., -0.0151, -0.1276,  0.0183],
         ...,
         [ 0.0815,  0.0265, -0.1012,  ...,  0.0748, -0.1033, -0.0103],
         [ 0.0337, -0.0482, -0.0812,  ...,  0.0457, -0.0733, -0.0114],
         [ 0.0326, -0.0313, -0.0514,  ...,  0.0472, -0.0762,  0.0344]],

        [[ 0.0805, -0.0372, -0.0449,  ..., -0.0173, -0.0270,  0.0051],
         [ 0.0997, -0.0174, -0.0306,  ..., -0.0302, -0.0752, -0.0308],
         [ 0.0734, -0.0427, -0.0462,  ...,  0.0573, -0.1426,  0.0064],
         ...,
         [ 0.0637,  0.0047, -0.0614,  ..., -0.0198, -0.0616, -0.0485],
         [ 0.0637,  0.0047, -0.0614,  ..., -0.0198, -0.0616, -0.0485],
         [ 0.0637,  0.0047, -0.0614,  ..., -0.0198, -0.0616, -0.0485]],

        [[ 0.1131, -0.0465, -0.0722,  ..., -0.0324, -0.0721,  0.0000],
         [ 0.1051, -0.0190, -0.0350,  ..., -0

...



  4%|▍         | 124/3081 [00:34<13:52,  3.55it/s]

tensor([[[ 0.1106, -0.2692, -0.4716,  ...,  1.0425, -0.0346,  0.0808],
         [ 0.1214, -0.3525, -0.5658,  ...,  0.9715, -0.0682, -0.0715],
         [ 0.0830, -0.4026, -0.5656,  ...,  1.1422, -0.0313,  0.0248],
         ...,
         [ 0.0950, -0.3964, -0.5422,  ...,  1.0609,  0.0089,  0.0049],
         [ 0.0390, -0.4321, -0.5075,  ...,  1.2473, -0.0349,  0.0604],
         [ 0.0255, -0.3978, -0.5132,  ...,  1.0008, -0.0377, -0.0216]],

        [[ 0.0241, -0.4669, -0.4651,  ...,  1.0525, -0.0334,  0.0030],
         [ 0.0668, -0.4103, -0.4857,  ...,  1.2305, -0.0577,  0.0030],
         [ 0.0312, -0.4412, -0.4956,  ...,  1.1038, -0.0800,  0.0240],
         ...,
         [ 0.0637, -0.2139, -0.3024,  ...,  0.5972, -0.0259, -0.0485],
         [ 0.0637, -0.2139, -0.3024,  ...,  0.5972, -0.0259, -0.0485],
         [ 0.0637, -0.2139, -0.3024,  ...,  0.5972, -0.0259, -0.0485]],

        [[ 0.0591, -0.3590, -0.5299,  ...,  1.0949, -0.0610, -0.0392],
         [ 0.0962, -0.4290, -0.4733,  ...,  1



  4%|▍         | 125/3081 [00:35<13:56,  3.54it/s]

  4%|▍         | 126/3081 [00:35<13:56,  3.53it/s]

tensor([[[ 0.0633, -0.4078, -0.5073,  ...,  1.0004, -0.0270, -0.0675],
         [ 0.0598, -0.4220, -0.4946,  ...,  0.9913, -0.0808, -0.0400],
         [ 0.0658, -0.5308, -0.5167,  ...,  1.2418, -0.0572, -0.0732],
         ...,
         [ 0.1236, -0.3783, -0.5911,  ...,  1.2572, -0.0247,  0.0030],
         [ 0.0649, -0.4065, -0.4727,  ...,  1.1561, -0.0179, -0.0099],
         [-0.0214, -0.3452, -0.5276,  ...,  1.1088, -0.0961,  0.0077]],

        [[ 0.0693, -0.4341, -0.5117,  ...,  1.0839, -0.0687, -0.1621],
         [ 0.0516, -0.4348, -0.5743,  ...,  1.3339, -0.0448,  0.0302],
         [ 0.0597, -0.4194, -0.5222,  ...,  1.1806, -0.0524, -0.0639],
         ...,
         [ 0.0637, -0.2146, -0.3036,  ...,  0.6002, -0.0255, -0.0485],
         [ 0.0637, -0.2146, -0.3036,  ...,  0.6002, -0.0255, -0.0485],
         [ 0.0637, -0.2146, -0.3036,  ...,  0.6002, -0.0255, -0.0485]],

        [[ 0.0905, -0.2954, -0.4461,  ...,  1.0384, -0.0946, -0.0350],
         [ 0.0748, -0.3966, -0.5271,  ...,  1

KeyboardInterrupt: 

In [53]:
dataset.label_alphabet.instance2index

{'S-CHEM': 1, 'B-CHEM': 2, 'I-CHEM': 3, 'O': 4, 'E-CHEM': 5}

In [6]:
dataset.word_alphabet.instance2index

{'</unk>': 0,
 'the': 1,
 'NS': 2,
 '0': 3,
 'B': 4,
 '-': 5,
 'protease': 6,
 'complex': 7,
 'is': 8,
 'essential': 9,
 'for': 10,
 'replication': 11,
 'of': 12,
 'dengue': 13,
 'virus': 14,
 ',': 15,
 'which': 16,
 'etiologic': 17,
 'agent': 18,
 'and': 19,
 'hemorrhagic': 20,
 'fevers': 21,
 'diseases': 22,
 'that': 23,
 'are': 24,
 'a': 25,
 'burden': 26,
 'tropical': 27,
 'subtropical': 28,
 'areas': 29,
 'world': 30,
 '.': 31,
 'active': 32,
 'form': 33,
 'linked': 34,
 'to': 35,
 '00': 36,
 'residues': 37,
 'cofactor': 38,
 'shows': 39,
 'highly': 40,
 'flexible': 41,
 'disordered': 42,
 'region': 43,
 '(': 44,
 's': 45,
 ')': 46,
 'responsible': 47,
 'its': 48,
 'high': 49,
 'propensity': 50,
 'aggregate': 51,
 'at': 52,
 'concentrations': 53,
 'necessary': 54,
 'NMR': 55,
 'spectroscopy': 56,
 'studies': 57,
 'or': 58,
 'crystallization': 59,
 'limited': 60,
 'proteolysis': 61,
 'this': 62,
 'enabled': 63,
 'us': 64,
 'obtain': 65,
 'folded': 66,
 'new': 67,
 'we': 68,
 'found

In [55]:
from lib.dataset import iter_seq_data
set([label for _, label in iter_seq_data(dataset.train_dir)])




0it [00:00, ?it/s]


94963it [00:00, 943172.64it/s]


190027it [00:00, 945759.21it/s]


267383it [00:00, 889455.19it/s]


336799it [00:00, 840945.69it/s]


403492it [00:00, 805931.79it/s]


496040it [00:00, 826002.18it/s]


595084it [00:00, 849490.64it/s]


697994it [00:00, 871754.97it/s]


798228it [00:00, 886229.68it/s]


900793it [00:01, 900204.93it/s]


903737it [00:01, 898912.50it/s]

{'B-CHEM', 'E-CHEM', False, 'I-CHEM', 'O', 'S-CHEM'}

In [40]:
dataset.word_alphabet.instances

['</unk>',
 'the',
 'NS',
 '0',
 'B',
 '-',
 'protease',
 'complex',
 'is',
 'essential',
 'for',
 'replication',
 'of',
 'dengue',
 'virus',
 ',',
 'which',
 'etiologic',
 'agent',
 'and',
 'hemorrhagic',
 'fevers',
 'diseases',
 'that',
 'are',
 'a',
 'burden',
 'tropical',
 'subtropical',
 'areas',
 'world',
 '.',
 'active',
 'form',
 'linked',
 'to',
 '00',
 'residues',
 'cofactor',
 'shows',
 'highly',
 'flexible',
 'disordered',
 'region',
 '(',
 's',
 ')',
 'responsible',
 'its',
 'high',
 'propensity',
 'aggregate',
 'at',
 'concentrations',
 'necessary',
 'NMR',
 'spectroscopy',
 'studies',
 'or',
 'crystallization',
 'limited',
 'proteolysis',
 'this',
 'enabled',
 'us',
 'obtain',
 'folded',
 'new',
 'we',
 'found',
 'from',
 'D',
 'E',
 'interacts',
 'directly',
 'strongly',
 'with',
 'domain',
 'proteolytic',
 'activity',
 'noncovalently',
 'binding',
 'was',
 'determined',
 'by',
 'rapid',
 'continuous',
 'fluorescence',
 'resonance',
 'energy',
 'transfer',
 'assay',
 'u

In [37]:
dataset.word_alphabet.size()

59281

In [39]:
len(dataset.word_alphabet.instances)

59280

In [106]:
# check Entity in RED text
for i in range(1000):
    print("=======================================================" * 2)
    sws = dataset.train_texts[i][2][0]
    new_sws = []
    for sw in sws:
        if not sw.startswith("▁"):
            new_sws[-1] += (sw + " ")
        else:
            new_sws.append(sw)
    for j, (word, label) in enumerate(zip(dataset.train_texts[i][0], dataset.train_texts[i][3])):
        if label == "O":
            print(new_sws[j], end="  ")
        else:
            print('\033[31m' + new_sws[j] + '\033[0m', end="  ")
    print("\n", end="")

▁DPP 0   ▁as  ▁a  ▁candidate  ▁gene  ▁for  ▁neurole p tic   ▁-   ▁induced  ▁tard ive   ▁dyskin es ia   ▁.   
▁we  ▁implemented   ▁a  ▁two  ▁-   ▁step  ▁approach  ▁to  ▁detect  ▁potential  ▁predictor   ▁gene  ▁variants  ▁for  ▁neurole p tic   ▁-   ▁induced  ▁tard ive   ▁dyskin es ia   ▁(  ▁TD   ▁)   ▁in  ▁schi zo ph re nic   ▁subjects  ▁.   
▁first  ▁,   ▁we  ▁screened   ▁associations   ▁by  ▁using  ▁a  ▁genome  ▁-   ▁wide  ▁(  ▁il lumin a   ▁HumanH ap C N V 000   ▁)   ▁SNP   ▁array  ▁in  ▁00   ▁j ap ane se   ▁schi zo ph re n ia   ▁patients  ▁with  ▁treatment  ▁-   ▁resistant   ▁TD   ▁and  ▁00   ▁j ap ane se   ▁schi zo ph re n ia   ▁patients  ▁without  ▁TD   ▁.   
▁next  ▁,   ▁we  ▁performed  ▁a  ▁replication  ▁analysis  ▁in  ▁00   ▁treatment  ▁-   ▁resistant   ▁TD   ▁and  ▁000   ▁non  ▁-   ▁TD   ▁subjects  ▁.   
▁an  ▁association  ▁of  ▁an  ▁SNP   ▁in  ▁the  ▁DPP 0   ▁(  ▁dip ept id yl   ▁peptidase   ▁-   ▁like   ▁protein  ▁-   ▁0   ▁)   ▁gene  ▁,   ▁rs 0 000 000   ▁,   ▁the  ▁most  ▁p

▁protective   ▁effect  ▁of  ▁hone y   ▁against  ▁cig ar et te   ▁smoke  ▁induced  ▁-   ▁impaired  ▁sexual  ▁behavior  ▁and  ▁fer til ity   ▁of  ▁male  ▁rats  ▁.   
▁cig ar et te   ▁smo k ing   ▁is  ▁associated  ▁with  ▁sexual  ▁dysfunction  ▁and  ▁impaired  ▁fer til ity   ▁in  ▁males   ▁.   
▁the  ▁aim  ▁of  ▁this  ▁study  ▁was  ▁to  ▁determine  ▁the  ▁potential  ▁protective   ▁effect  ▁of  ▁hone y   ▁against  ▁the  ▁toxic  ▁effect  ▁of  ▁cig ar et te   ▁smoke  ▁(  ▁CS   ▁)   ▁on  ▁sexual  ▁behavior  ▁and  ▁fer til ity   ▁of  ▁male  ▁rats  ▁.   
▁th ir ty   ▁-   ▁two  ▁adult  ▁spra gu e   ▁-   ▁daw le y   ▁rats  ▁were  ▁randomly   ▁divided  ▁into  ▁four  ▁groups  ▁(  ▁0   ▁rats  ▁/   ▁group  ▁)   ▁as  ▁control  ▁,   ▁hone y   ▁(  ▁H  ▁)   ▁,   ▁CS   ▁and  ▁H  ▁plus  ▁CS   ▁(  ▁H  ▁+  ▁CS   ▁)   ▁groups  ▁.   
▁rats  ▁in  ▁control  ▁and  ▁CS   ▁groups  ▁received  ▁oral  ▁administration  ▁of  ▁dist ill ed   ▁water  ▁daily  ▁while  ▁rats  ▁in  ▁H  ▁and  ▁H  ▁+  ▁CS   ▁groups  ▁received  ▁

▁two  ▁new  ▁spir ost an ol s   ▁and  ▁a  ▁new  ▁fur ost an ol   ▁,   ▁rein oc ar n oside   ▁A  ▁(  ▁0   ▁)   ▁,   ▁B  ▁(  ▁0   ▁)   ▁and  ▁C  ▁(  ▁0   ▁)   ▁,   ▁were  ▁isolated  ▁from  ▁the  ▁roots   ▁of  ▁reine c k ia   ▁carne a   ▁,   ▁together  ▁with  ▁two  ▁known  ▁compounds  ▁,   ▁(  ▁00 S   ▁)   ▁-   ▁0 β , 0 β , 0 β   ▁-   ▁trihydroxy s pir ost an   ▁-   ▁0 β   ▁-   ▁yl   ▁-   ▁O  ▁-   ▁β  ▁-   ▁D  ▁-   ▁gluco pyr an oside   ▁(  ▁0   ▁)   ▁,   ▁kit i gen in   ▁-   ▁0 β   ▁-   ▁O  ▁-   ▁β  ▁-   ▁D  ▁-   ▁gluco pyr an oside   ▁(  ▁0   ▁)   ▁.   
▁the  ▁structures  ▁of  ▁three  ▁new  ▁compounds  ▁were  ▁elucidated  ▁by  ▁spectroscopic  ▁methods  ▁including  ▁0   ▁-   ▁D  ▁NMR  ▁,   ▁0   ▁-   ▁D  ▁NMR  ▁and  ▁MS   ▁spectrums   ▁,   ▁and  ▁their  ▁anticancer  ▁activities  ▁were  ▁evaluated  ▁by  ▁MT T   ▁method  ▁.   
▁seas on al   ▁variation  ▁in  ▁biomarkers   ▁in  ▁blue  ▁mus s el   ▁(  ▁mytil us   ▁ed ul is   ▁)   ▁,   ▁ice land ic   ▁scall op   ▁(  ▁chl am y s   ▁island ic a  

▁in  ▁the  ▁present  ▁work  ▁,   ▁the  ▁lethality   ▁percent ile s   ▁including  ▁median   ▁lethal  ▁doses  ▁(  ▁LD 00   ▁)   ▁,   ▁accumulation  ▁,   ▁distribution  ▁and  ▁toxico kin etic s   ▁of  ▁alumin um   ▁in  ▁the  ▁liver  ▁,   ▁kidney  ▁,   ▁intestine  ▁,   ▁brain  ▁and  ▁serum  ▁of  ▁male  ▁albi no   ▁rats  ▁,   ▁following  ▁a  ▁single  ▁oral  ▁administration  ▁were  ▁studied  ▁throughout  ▁0   ▁,   ▁0   ▁,   ▁0   ▁,   ▁00   ▁and  ▁00   ▁days  ▁.   
▁the  ▁estimated  ▁LD 00   ▁at  ▁00   ▁h  ▁was  ▁0.00   ▁g  ▁al  ▁/   ▁kg   ▁body  ▁weight  ▁(  ▁b. w t .   ▁)   ▁.   
▁the  ▁utilized   ▁dose  ▁of  ▁al  ▁was  ▁0   ▁/   ▁00   ▁LD 00   ▁(  ▁0.00   ▁g  ▁al  ▁/   ▁kg   ▁b. w t .   ▁)   ▁.   
▁alumin um   ▁residues  ▁,   ▁in  ▁al  ▁-   ▁treated  ▁rats  ▁,   ▁were  ▁significantly  ▁decreased  ▁in  ▁response  ▁to  ▁the  ▁experimental  ▁periods   ▁and  ▁were  ▁negatively   ▁correlated  ▁with  ▁time  ▁.   
▁in  ▁addition  ▁,   ▁the  ▁hepatic  ▁,   ▁renal  ▁,   ▁intestinal  ▁,   ▁brain  ▁a

▁in  ▁vitro  ▁antif un g al   ▁activity  ▁is  ▁evaluated  ▁in  ▁order  ▁to  ▁identify  ▁new  ▁means   ▁that  ▁could  ▁be  ▁helpful   ▁in  ▁the  ▁prevention   ▁of  ▁cont amin ation   ▁in  ▁indo or   ▁environments   ▁.   
▁carbon  ▁sources   ▁and  ▁medium  ▁pH  ▁affects   ▁the  ▁growth  ▁of  ▁withan ia   ▁som ni fer a   ▁(  ▁L.   ▁)   ▁dun al   ▁ad vent it i ous   ▁roots   ▁and  ▁withan ol ide   ▁A  ▁production  ▁.   
▁this  ▁work  ▁deal s   ▁with  ▁op tim is ation   ▁of  ▁ad vent it i ous   ▁root  ▁suspension  ▁culture  ▁of  ▁withan ia   ▁som ni fer a   ▁(  ▁L.   ▁)   ▁dun al   ▁(  ▁solan ace ae   ▁)   ▁for  ▁the  ▁production  ▁of  ▁bioma s s   ▁and  ▁withan ol ide   ▁A  ▁and  ▁we  ▁investigated  ▁the  ▁effect  ▁of  ▁carbon  ▁source  ▁and  ▁the  ▁initial  ▁medium  ▁pH  ▁on  ▁growth  ▁and  ▁production  ▁of  ▁withan ol ide   ▁A  ▁in  ▁ad vent it i ous   ▁root  ▁cultures   ▁of  ▁withan ia   ▁som ni fer a   ▁.   
▁A  ▁0   ▁%   ▁sucrose  ▁concentration  ▁was  ▁found  ▁to  ▁be  ▁best  ▁for  ▁

...

▁however  ▁,   ▁in  ▁the  ▁MetS   ▁subjects  ▁,   ▁00   ▁%   ▁of  ▁NH W   ▁,   ▁00   ▁%   ▁of  ▁NH B   ▁,   ▁and  ▁00   ▁%   ▁of  ▁MA   ▁were  ▁not  ▁in  ▁the  ▁IR  ▁ter til e   ▁and  ▁in  ▁the  ▁IR  ▁subjects  ▁,   ▁00   ▁%   ▁of  ▁NH W   ▁,   ▁00   ▁%   ▁NH B   ▁,   ▁and  ▁00   ▁%   ▁of  ▁MA   ▁did  ▁not  ▁have  ▁MetS   ▁.   
▁no  ▁relationship  ▁was  ▁found  ▁between  ▁beta  ▁cell  ▁function  ▁and  ▁MetS   ▁.   
▁although  ▁IR  ▁is  ▁a  ▁risk  ▁factor  ▁for  ▁MetS   ▁,   ▁IR  ▁is  ▁neither  ▁necessary  ▁nor  ▁required  ▁for  ▁MetS   ▁.   
▁an  ▁in  ▁vitro  ▁study  ▁on  ▁the  ▁DNA  ▁dam ag ing   ▁effects  ▁of  ▁phytochemical s   ▁partially  ▁isolated  ▁from  ▁an  ▁extract  ▁of  ▁glin us   ▁lo to ides   ▁.   
▁an  ▁extract  ▁of  ▁glin us   ▁lo to ides   ▁,   ▁a  ▁medicinal  ▁plant  ▁used  ▁in  ▁afr ic a   ▁and  ▁asia   ▁for  ▁various  ▁therapeutic  ▁purposes   ▁,   ▁was  ▁recently  ▁shown  ▁to  ▁cause  ▁DNA  ▁damage  ▁in  ▁vitro  ▁.   
▁to  ▁further  ▁explore  ▁the  ▁potential  ▁genot

▁taken   ▁together  ▁,   ▁this  ▁study  ▁suggests  ▁that  ▁top ical   ▁formulations   ▁such  ▁as  ▁cre am s   ▁including  ▁SL N s   ▁with  ▁QNT 0   ▁-   ▁00   ▁might  ▁be  ▁appropriate  ▁carriers   ▁for  ▁skin  ▁whiten ing   ▁agents  ▁.   
▁formul as   ▁used  ▁by  ▁ti beta n   ▁doct ors   ▁at  ▁men  ▁-   ▁tse e   ▁-   ▁kh ang   ▁in  ▁india   ▁for  ▁the  ▁treatment  ▁of  ▁neurop s y chi a tric   ▁disorders  ▁and  ▁their  ▁correlation  ▁with  ▁pharmacological  ▁data  ▁.   
▁the  ▁aim  ▁of  ▁the  ▁present  ▁study  ▁was  ▁to  ▁identify  ▁formul as   ▁used  ▁at  ▁men  ▁-   ▁tse e   ▁-   ▁kh ang   ▁(  ▁ti beta n   ▁medical  ▁and  ▁astr ological   ▁institut e   ▁)   ▁,   ▁india   ▁,   ▁for  ▁the  ▁treatment  ▁of  ▁neurop s y chi a tric   ▁disorders  ▁and  ▁to  ▁compare  ▁the  ▁ti beta n   ▁us age   ▁of  ▁particular  ▁ing re di ent s   ▁with  ▁pharmacological  ▁data  ▁from  ▁the  ▁sci ent if ic   ▁database  ▁.   
▁using  ▁eth n ographic   ▁methods  ▁,   ▁five  ▁doct ors   ▁were  ▁selected  ▁an

▁the  ▁remaining   ▁0   ▁%   ▁was  ▁made  ▁up  ▁by  ▁cyan idin   ▁-   ▁0   ▁-   ▁O  ▁-   ▁[  ▁0   ▁-   ▁O  ▁-   ▁(  ▁0   ▁-   ▁O  ▁-   ▁E  ▁-   ▁caff e oyl   ▁-   ▁O  ▁-   ▁α  ▁-   ▁rham no pyr an osyl   ▁)   ▁-   ▁β  ▁-   ▁gluco pyr ano   ▁]   ▁-   ▁0   ▁-   ▁O  ▁-   ▁β  ▁-   ▁gluco pyr an oside   ▁and  ▁pel ar gon idin   ▁-   ▁0   ▁-   ▁O  ▁-   ▁[  ▁0   ▁-   ▁O  ▁-   ▁(  ▁0   ▁-   ▁O  ▁-   ▁E  ▁-   ▁p  ▁-   ▁coum ar oyl   ▁-   ▁O  ▁-   ▁α  ▁-   ▁rham no pyr an osyl   ▁)   ▁-   ▁β  ▁-   ▁gluco pyr ano   ▁]   ▁-   ▁0   ▁-   ▁O  ▁-   ▁β  ▁-   ▁gluco pyr an oside   ▁.   
▁an  ▁op to gene tic   ▁approach  ▁in  ▁epile p s y   ▁.   
▁op to gene tic   ▁tool s   ▁comprise  ▁a  ▁variety  ▁of  ▁different  ▁light  ▁-   ▁sensitive  ▁proteins  ▁from  ▁single  ▁-   ▁cell  ▁organism s   ▁that  ▁can  ▁be  ▁expressed  ▁in  ▁mammalian  ▁neurons  ▁and  ▁effectively   ▁control  ▁their  ▁excit ability   ▁.   
▁two  ▁main  ▁classes   ▁of  ▁op to gene tic   ▁tool s   ▁allow  ▁to  ▁either  ▁depolar ize   ▁or

▁in  ▁addition  ▁,   ▁the  ▁isoco um ar in   ▁(  ▁0   ▁)   ▁inhibited  ▁the  ▁growth  ▁of  ▁sta phyl ococcus   ▁aure us   ▁with  ▁the  ▁IC00   ▁value  ▁of  ▁0 0.00   ▁µ g   ▁mL   ▁(  ▁-   ▁0   ▁)   ▁.   
▁transfer  ▁of  ▁a  ▁two  ▁-   ▁ti er ed   ▁ker at in ocyte   ▁assay  ▁:   ▁IL  ▁-   ▁00   ▁production  ▁by  ▁NCT C 0 000   ▁to  ▁determine  ▁the  ▁skin  ▁sens it izing   ▁capacity  ▁and  ▁epidermal  ▁equivalent  ▁assay  ▁to  ▁determine  ▁sens it ize r   ▁potency  ▁.   
▁at  ▁present  ▁,   ▁the  ▁identification  ▁of  ▁potentially  ▁sens it izing   ▁chemicals   ▁is  ▁carried  ▁out  ▁using  ▁animal  ▁models  ▁.   
▁however  ▁,   ▁it  ▁is  ▁very  ▁important  ▁from  ▁eth ical   ▁,   ▁safety  ▁and  ▁econ omic   ▁point  ▁of  ▁view  ▁to  ▁have  ▁biological  ▁markers  ▁to  ▁discri min ate   ▁aller g y   ▁and  ▁ir rit ation   ▁events  ▁,   ▁and  ▁to  ▁be  ▁able  ▁to  ▁classif y   ▁sens it iz ers   ▁according  ▁to  ▁their  ▁potency  ▁,   ▁without  ▁the  ▁use  ▁of  ▁animals  ▁.   
▁within  ▁the  

▁in  ▁vitro  ▁investigation  ▁of  ▁the  ▁inhibition  ▁of  ▁00   ▁(  ▁S  ▁)   ▁-   ▁protop an ax a tri ol   ▁(  ▁ppt   ▁)   ▁towards   ▁UG T 0 A 0   ▁and  ▁UG T 0 B 0   ▁was  ▁carried  ▁out  ▁.   
▁the  ▁results  ▁showed  ▁that  ▁ppt   ▁exhibited  ▁strong  ▁noncompetitive   ▁inhibition  ▁towards   ▁UG T 0 A 0   ▁and  ▁competitive   ▁inhibition  ▁towards   ▁UG T 0 B 0   ▁.   
▁the  ▁inhibition  ▁kinetic  ▁parameters  ▁(  ▁ki   ▁)   ▁were  ▁calculated  ▁to  ▁be  ▁0.0   ▁and  ▁0.0   ▁μ M   ▁for  ▁UG T 0 A 0   ▁and  ▁UG T 0 B 0   ▁,   ▁respectively  ▁.   
▁using  ▁the  ▁maximum  ▁plasma  ▁concentration  ▁of  ▁ppt   ▁,   ▁the  ▁alteration   ▁of  ▁area  ▁under  ▁the  ▁concentration  ▁-   ▁time  ▁curve  ▁was  ▁calculated  ▁to  ▁be  ▁00   ▁%   ▁and  ▁00   ▁%   ▁respectively  ▁for  ▁UG T 0 A 0   ▁-   ▁mediated  ▁and  ▁UG T 0 B 0   ▁-   ▁mediated  ▁metabolism  ▁.   
▁however  ▁,   ▁given  ▁that  ▁the  ▁varied  ▁contribution  ▁of  ▁these  ▁two  ▁UG T   ▁isoforms   ▁towards   ▁drug  ▁metabolism  ▁a

In [110]:
!python3.7 ner/train.py --config config/csw.4k_16k.config

============= load config file ==================
train_dir=/home/sekine/SubwordSequenceLabeling/Repository/Chemdner/train.bioes: <class 'str'>
dev_dir=/home/sekine/SubwordSequenceLabeling/Repository/Chemdner/valid.bioes: <class 'str'>
test_dir=/home/sekine/SubwordSequenceLabeling/Repository/Chemdner/test.bioes: <class 'str'>
model_dir=/home/sekine/SubwordSequenceLabeling/Repository/NERModel/csw4k_16k.lm20k: <class 'str'>
lm_dir=/home/sekine/SubwordSequenceLabeling/Repository/LargeCorpus/large_corpus_20k.txt: <class 'str'>
pretrain_model_dir=/home/sekine/SubwordSequenceLabeling/Repository/LanguageModel/csw4k_16k.lm20k: <class 'str'>
word_emb_dir=/home/sekine/SubwordSequenceLabeling/Repository/GloVe/gv.d50: <class 'str'>
sw_num=2: <class 'int'>
sentence_piece_dirs=[/home/sekine/SubwordSequenceLabeling/Repository/SentencePiece/sp4000.model,/home/sekine/SubwordSequenceLabeling/Repository/SentencePiece/sp16000.model]: <class 'list'>
word_emb_dim=50: <class 'int'>
char_emb_dim=30: <class 'i

torch.Size([100, 78, 200])
torch.Size([100, 108, 200])
torch.Size([100, 209, 200])
torch.Size([100, 73, 200])
torch.Size([100, 63, 200])
torch.Size([100, 90, 200])
torch.Size([100, 69, 200])
torch.Size([100, 125, 200])
torch.Size([100, 88, 200])
torch.Size([100, 77, 200])
torch.Size([100, 81, 200])
torch.Size([100, 156, 200])
torch.Size([100, 91, 200])
torch.Size([100, 74, 200])
torch.Size([100, 171, 200])
torch.Size([100, 95, 200])
torch.Size([100, 77, 200])
torch.Size([100, 95, 200])
torch.Size([100, 133, 200])
torch.Size([100, 66, 200])
torch.Size([100, 96, 200])
torch.Size([100, 80, 200])
torch.Size([100, 89, 200])
torch.Size([100, 95, 200])
torch.Size([100, 131, 200])
torch.Size([100, 113, 200])
torch.Size([100, 66, 200])
torch.Size([100, 93, 200])
torch.Size([100, 123, 200])
torch.Size([100, 101, 200])
torch.Size([100, 77, 200])
torch.Size([100, 119, 200])
torch.Size([100, 65, 200])
torch.Size([100, 84, 200])
torch.Size([100, 108, 200])
torch.Size([100, 69, 200])
torch.Size([100,

In [111]:
class Trainer:
    epoch
    batch_size
    optimizer
    losses
    
    def initPerEpoch():
        pass
    
    def initPerBatch():
        pass

NameError: name 'epoch' is not defined

In [112]:
class InputFeature:
    """一文に対する構造体"""
    text_id: int
    text = "I have a pen"
    word = ["I", "have", "a", "pen"]
    char = [["I"], ["h", "a", "v", "e"], ["a"], ["pen"]]
    sws = [["I", "ha", "ve", "a", "pe", "n"], ["I", "hav", "e", "a", "pen"]]
    label = ["O", "S", "O", "O"]
    word_tensor
    char_tensor
    sws_tensor
    
    def loadFromText():
        pass
    
    def batchify():
        pass
    
    def generateIDs():
        pass

NameError: name 'texts' is not defined

In [ ]:
input_batch_dict = {
    words: Tensor()
    chars
    sws
    labels
    word_length
    char_length
    sw_lengths
    word_recover
    char_recover
    sw_recover
}

In [113]:
############ Language Modelの学習と予測 ###############

def preprocess():
    word_alphabet = Alphabet()
    char_alphabet = Alphabet()
    sw_alphabets = [Alphabet() for i in range(config.sw_num)]
    #build_alphabet()
    for text, label in iter_file():
        update_word_alphabet(words)
        update_char_alphabet(words)
        for i in range(config.sw_num):
            update_sw_alphabet(words)
    
    input_features = []
    for text, label in iter_file():
        words = tokenize(text)    
        input_feature = InputFeature(words)
        input_feature.generate_IDs()
        input_features.append(InputFeature(words))
    
def train():
    preprocess()
    build_pretrain_emb()    
    model = LanguageModel(config)
    for epoch in range(epochs):
        init_per_epoch()
        for batch in range(batch_size):
            init_per_batch()
            input_batch_dic = batchify(input_features[start:end])
            f_pred_ids, b_pred_ids = language_model.get_bidirectional_ids(input_batch_dic)
            f_correct_ids, b_correct_ids = get_bidirectional_correct_ids(input_features[start:end])
            loss += loss_func(f_pred_ids, f_correct_ids)
            loss += loss_func(b_pred_ids, b_correct_ids)
            loss.backward()
            optimizer.step()
            model.zero_grad()

def predict(text):
    word_alphabet, char_alphabet, sw_alphabets = load_alphabet()
    input_feature = InputFeature.load_from_text(text)
    input_feature.generateIDs()
    input_tensor_dic = get_input_dic(input_features)
    model = load_model()
    f_pred_ids, b_pred_ids = model.get_bidirectional_ids(input_tensor_dic)
    pred_next_words = get_text_from_pred_IDs(f_pred_ids)
    pred_before_words = get_text_from_pred_IDs(b_pred_ids)

In [ ]:
################ NER Modelの学習と予測と評価 ######################
def preprocess():
    # ========= build_alphabet() ===========
    word_alphabet = Alphabet()
    char_alphabet = Alphabet()
    sw_alphabets = [Alphabet() for i in range(config.sw_num)]
    for text, label in iter_file():
        update_word_alphabet(words)
        update_char_alphabet(words)
        for i in range(config.sw_num):
            update_sw_alphabet(words)
    # ========== get_input_features() =================
    input_features = []
    for text, label in iter_file():
        words = tokenize(text)    
        input_feature = InputFeature(words, label)
        input_feature.generate_IDs()
        input_features.append(InputFeature(words))

def train():
    build_alphabet()
    input_features = get_input_features()
    
    
def predict():
    
def evaluate():
    


In [11]:
class Config:
    
    def __init__(self):
        self.sentencepiece_models = []

    def load_from_config_file(self, config_file):
        config = config_file_to_dict(config_file)
        self.__dict__.update(config)
        self.sw_alphabet_list = [Alphabet(f'sw{i}') for i in range(self.sw_num)]
        for sp_path in self.sentence_piece_dirs:
            sp = ChemSentencePiece()
            sp.load(sp_path)
            self.sentencepiece_models.append(sp)

# mainの処理を以下に書く

In [52]:
from tqdm import tqdm

from lib.config import config_file_to_dict
from lib.alphabet import Alphabet
from lib.function import tokenize
from chem_sentencepiece.chem_sentencepiece import ChemSentencePiece

# language model train()
config = Config()
config.load_from_config_file("config/csw.4k_16k.1k.config")
word_alphabet = Alphabet("word")
char_alphabet = Alphabet("char")
sw_alphabets = [Alphabet(f"sw{i}") for i in range(config.sw_num)]
input_features = build_alphabet_and_input_features(config.lm_dir, mode="lm")












0it [00:00, ?it/s]










11it [00:00, 106.57it/s]

============= load config file ==================
train_dir=/home/sekine/SubwordSequenceLabeling/Repository/Chemdner/train.bioes: <class 'str'>
dev_dir=/home/sekine/SubwordSequenceLabeling/Repository/Chemdner/valid.bioes: <class 'str'>
test_dir=/home/sekine/SubwordSequenceLabeling/Repository/Chemdner/test.bioes: <class 'str'>
model_dir=/home/sekine/SubwordSequenceLabeling/Repository/NERModel/csw4k_16k.lm1k: <class 'str'>
lm_dir=/home/sekine/SubwordSequenceLabeling/Repository/LargeCorpus/large_corpus_1k.txt: <class 'str'>
pretrain_model_dir=/home/sekine/SubwordSequenceLabeling/Repository/LanguageModel/csw4k_16k.lm1k: <class 'str'>
word_emb_dir=/home/sekine/SubwordSequenceLabeling/Repository/GloVe/gv.d50: <class 'str'>
sw_num=2: <class 'int'>
sentence_piece_dirs=[/home/sekine/SubwordSequenceLabeling/Repository/SentencePiece/sp4000.model,/home/sekine/SubwordSequenceLabeling/Repository/SentencePiece/sp16000.model]: <class 'list'>
word_emb_dim=50: <class 'int'>
char_emb_dim=30: <class 'int'












22it [00:00, 107.34it/s]










31it [00:00, 101.42it/s]










42it [00:00, 102.50it/s]










53it [00:00, 103.05it/s]










67it [00:00, 108.54it/s]










82it [00:00, 114.26it/s]










95it [00:00, 115.90it/s]










107it [00:00, 115.80it/s]










128it [00:01, 124.89it/s]










147it [00:01, 130.66it/s]










164it [00:01, 133.64it/s]










181it [00:01, 135.73it/s]










198it [00:01, 137.73it/s]










215it [00:01, 139.59it/s]










232it [00:01, 139.24it/s]










248it [00:01, 139.92it/s]










264it [00:01, 140.04it/s]










279it [00:01, 140.27it/s]










294it [00:02, 139.46it/s]










309it [00:02, 139.42it/s]










324it [00:02, 139.31it/s]










339it [00:02, 139.75it/s]










354it [00:02, 137.15it/s]










367it [00:02, 136.67it/s]










380it [00:02, 136.01it/s]










393it [00:02, 135.64it/s]










412it [00:02, 137.43it/s]










427it [00:03, 136.91it/s

In [51]:
import re

from lib.function import normalize_word, cammel_normalize_word

def iter_text_data(file_name):
    """textデータを行ごとにtokenizeしたlistで返す。"""
    texts = open(file_name,'r').read()
    for i, lines in tqdm(enumerate(texts.split("\n"))):
        for line in lines.split(". "):
            yield tokenize(line) + ["."], False

def build_alphabet_and_input_features(input_file, mode="ner"):
    input_features = []
    if mode == "ner":
        _iter = iter_seq_data(input_file)
    elif mode == "lm":
        _iter = iter_text_data(input_file)
    for i, (words, labels) in enumerate(_iter):
        input_feature = InputFeature(sw_num=len(config.sentencepiece_models))
        
        ### normalize ###
        if config.number_normalized:
            words = [normalize_word(word) for word in words]
        if config.cammel_normalized:
            words = [normalize_word(word) for word in words]
        for word in words:
            word_alphabet.add(word)
            input_feature.words.append(word)
            for char in word:
                char_alphabet.add(char)
            input_feature.chars.append([char for char in word])
            

            for sw_ix, sp in enumerate(config.sentencepiece_models):
                for sw in sp.tokenize(word):
                    sw_alphabets[sw_ix].add(sw)
                    input_feature.sws[sw_ix].append(sw)
                    input_feature.sw_fmasks[sw_ix].append(0)
                    input_feature.sw_bmasks[sw_ix].append(0)
                input_feature.sw_fmasks[sw_ix][-1] = 1
                input_feature.sw_bmasks[sw_ix][0] = 1
        if mode == "ner":
            for label in set(labels):
                label_alphabet.add(label)
            input_feature.labels = labels
        input_features.append(input_feature)
    
    return input_features

In [56]:
class InputFeature():
    
    def __init__(self, sw_num):
        self.words = []
        self.chars = []
        self.sws = [[] for i in range(sw_num)]
        self.sw_fmasks = [[] for i in range(sw_num)]
        self.sw_bmasks = [[] for i in range(sw_num)]
        self.labels = []
        self.word_IDs = []
        self.char_IDs = []
        self.sw_IDs = [[] for i in range(sw_num)]
        self.label_IDs = []
    
    def generateIDs(self, word_alphabet, char_alphabet, sw_alphabets):
        self.word_IDs = [word_alphabet.get_index(word) for word in self.words]
        self.char_IDs = [[char_alphabet.get_index(c) for c in char] for char in self.chars]
        self.sw_IDs = [[sw_alphabets[i].get_index(sw) for sw in self.sws] for i in range(config.sw_num)]

In [55]:
input_features[0].words

['The',
 'NS',
 '0',
 'B',
 '-',
 'NS',
 '0',
 'protease',
 'complex',
 'is',
 'essential',
 'for',
 'the',
 'replication',
 'of',
 'dengue',
 'virus',
 ',',
 'which',
 'is',
 'the',
 'etiologic',
 'agent',
 'of',
 'dengue',
 'and',
 'hemorrhagic',
 'fevers',
 ',',
 'diseases',
 'that',
 'are',
 'a',
 'burden',
 'for',
 'the',
 'tropical',
 'and',
 'subtropical',
 'areas',
 'of',
 'the',
 'world',
 '.']